In [ ]:
HTML(read(open("style.css"), String))

# Engine Games

This notebook contains functions for 2 engines to play against each other. 

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("NBInclude")
using NBInclude

## Engines

In [ ]:
@nbinclude("RandomChess.ipynb")

In [ ]:
@nbinclude("Minimax.ipynb")

In [ ]:
@nbinclude("AlphaBetaPruning.ipynb")

In [ ]:
@nbinclude("IterativeDeepening.ipynb")

## Utility

In [ ]:
@nbinclude("Utility.ipynb")

In [ ]:
@nbinclude("PGN_Export.ipynb")

In [ ]:
@nbinclude("PGN_Import.ipynb")

## Players

The `playWhiteMove` function takes in 4 arguments.
1. `game` is the current state of type `Game`
1. `current_boardscore` is the static evaluation of the position from the perspective of the moving player
1. `current_hash` is the hash of the current position
1. `cache`  is a dictionary which stores the calculated values

The function makes a move for the white player and returns the new score and hash of the position.

In [ ]:
function playWhiteMove(game::Game, current_boardscore::Int64, current_hash::UInt64,  cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(board(game), currentcurrent_boardscorescore, 4, cache)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(board(game), current_boardscore, 4, cache)

    # ITERATIVE DEEPENING ENGINE
    eval, move = iterativeDeepening(board(game), current_boardscore, current_hash, 4, cache)

    #----------------------------------------------------------------
    
    println("Evaluation of engine: $eval")
    println("Current board score: $current_boardscore")
    println("Current entries in Cache: $(length(Cache))")
    clearCache(board(game), move)
    current_boardscore = evaluate_move(board(game), move, current_boardscore)
    current_hash = zobrist_hash(board(game), current_hash, move)
    domove!(game, move)
    return current_boardscore, current_hash
end

The `playBlackMove` function takes in 4 arguments.
1. `game` is the current state of type `Game`
1. `current_boardscore` is the static evaluation of the position from the perspective of the moving player
1. `current_hash` is the hash of the current position
1. `cache`  is a dictionary which stores the calculated values

The function makes a move for the black player and returns the new score and hash of the position.

In [ ]:
function playBlackMove(game::Game, current_boardscore::Int64, current_hash::UInt64, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(board(game), currentcurrent_boardscorescore, 4, cache)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(board(game), current_boardscore, 4, cache)

    # ITERATIVE DEEPENING ENGINE
    eval, move = iterativeDeepening(board(game), current_boardscore, current_hash, 2, cache)

    #----------------------------------------------------------------
    
    println("Evaluation of engine: $eval")
    println("Current board score: $current_boardscore")
    println("Current entries in Cache: $(length(Cache))")
    clearCache(board(game), move)
    current_boardscore = evaluate_move(board(game), move, current_boardscore)
    current_hash = zobrist_hash(board(game), current_hash, move)
    domove!(game, move)
    return current_boardscore, current_hash
end

## Game

The `playGame` function takes in optional 1 arguments. 
1. `pgnFile` is optional and default is nothing. 'pgnFile' is the name of the pgn file to be opened. If nothing is passed, the function will generate a new game.

The function plays a game between the white and black engines and returns the final state of the game.

In [ ]:
function playGame(pgnFile=nothing)
    # Setup Board
    if pgnFile == nothing
        # new Game
        println("Initializing new Game...")
        game = Game()
        setGameHeaders(game)
    else
        # continue saved game
        println("Opening $(pgnFile) ...")
        game = openPGNtoGame(pgnFile)
        if game == nothing
            println("No such file found")
            return
        end
        toend!(game)
        println(game)
    end
    
    current_boardscore = evaluate_position(board(game))
    current_hash = zobrist_hash(board(game))
    
    cacheWhite = initCache()
    cacheBlack = initCache()
    moveCount = 1
    while true
        # White Move
        
        println("Move $(moveCount)")
        println("White")
        
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                current_boardscore, current_hash = playWhiteMove(game, current_boardscore, current_hash, cacheWhite)
            end
        else
            break
        end
        
        # Black Move
        println("Black")
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                current_boardscore, current_hash = playBlackMove(game, current_boardscore, current_hash, cacheBlack)
            end
        else
            break
        end
        moveCount += 1
    end
    setWin(game)
    saveGameToPGN(game)
    return game
end

In [ ]:
playGame()